In [1]:
import numpy as np
import pandas as pd

In [2]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [3]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA3c/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'oth'],
      dtype='object')

In [4]:
mineralogy = mineralogy.dropna()

In [5]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [6]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["oth"], axis=1))

In [7]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [8]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
11610,67.038949,0.356336,17.481964,0.059306,3.249495,0.051174,1.253712,4.090304,3.457030,2.961731
11611,67.031897,0.356442,17.484426,0.059263,3.250803,0.051180,1.254825,4.093602,3.456848,2.960714
12060,67.052490,0.356141,17.477151,0.059397,3.247035,0.051167,1.251608,4.083952,3.457365,2.963695
12061,67.045525,0.356247,17.479569,0.059355,3.248339,0.051172,1.252714,4.087203,3.457183,2.962692
12062,67.038407,0.356355,17.482047,0.059312,3.249666,0.051177,1.253841,4.090530,3.456999,2.961666


In [9]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
11610,67.038949,0.356336,17.481964,0.059306,3.249495,0.051174,1.253712,4.090304,3.457030,2.961731
11611,67.031897,0.356442,17.484426,0.059263,3.250803,0.051180,1.254825,4.093602,3.456848,2.960714
12060,67.052490,0.356141,17.477151,0.059397,3.247035,0.051167,1.251608,4.083952,3.457365,2.963695
12061,67.045525,0.356247,17.479569,0.059355,3.248339,0.051172,1.252714,4.087203,3.457183,2.962692
12062,67.038407,0.356355,17.482047,0.059312,3.249666,0.051177,1.253841,4.090530,3.456999,2.961666
...,...,...,...,...,...,...,...,...,...,...
218877,59.959647,0.686645,17.257058,0.333523,5.808659,0.097062,3.894096,6.598686,2.998763,2.365861
218878,59.959437,0.686647,17.257098,0.333513,5.808693,0.097061,3.894156,6.598797,2.998757,2.365840
218879,59.959252,0.686649,17.257135,0.333504,5.808722,0.097061,3.894208,6.598897,2.998752,2.365821
218880,59.959093,0.686650,17.257169,0.333495,5.808745,0.097061,3.894251,6.598984,2.998748,2.365804


In [10]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45000:90000]
mineralogy3 = mineralogy.iloc[90000:140000]

In [11]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA3c/area3c_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA3c/area3c_2.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy3, "../_CIPW/CIPW/AREA3c/area3c_3.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [13]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA3c/area3c_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA3c/area3c_2_results_excel.txt")
df_final3 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA3c/area3c_3_results_excel.txt")

In [14]:
df_final = pd.concat([df_final1, df_final2, df_final3])

In [15]:
df_final

,Q,P,K
SAMPLE,,,
11610,22.488916,46.616341,20.426281
11611,22.477034,46.631338,20.420101
12060,22.511618,46.587428,20.438211
12061,22.499987,46.602105,20.432082
12062,22.487959,46.617251,20.425899
...,...,...,...
218877,10.534490,49.474681,16.517460
218878,10.534085,49.474847,16.517341
218879,10.533783,49.474956,16.517215


## QAPF classification

In [16]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [17]:
df_final["QAPF"].unique()

array(['granodiorite', 'quartz monzodiorite\nquartz monzogabbro',
       'monzo granite'], dtype=object)

In [18]:
df_final.to_excel("../_CIPW/CIPW/AREA3c/qapf.xlsx", index=True)

-------

In [19]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA3c/QAPF_counts.xlsx")

-----

In [20]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area3_c.xlsx", index_col=0, usecols = [0, 1, 2, 3, 4, 29])

In [21]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
63,718167.786554,7.766777e+06,53.0,W,8.0,36.451111,31.480546,29.802500,monzo granite
180,696837.389941,7.761141e+06,53.0,W,8.0,35.918400,25.049310,34.895732,monzo granite
339,717014.806153,7.772283e+06,53.0,W,8.0,40.499596,32.471476,21.361427,monzo granite
870,730159.789021,7.775423e+06,53.0,W,8.0,34.233700,29.296636,30.826628,monzo granite
2209,725221.207433,7.726356e+06,53.0,W,8.0,27.880737,39.617438,26.380274,monzo granite
2365,698199.574434,7.760355e+06,53.0,W,8.0,25.954517,37.070517,29.559522,monzo granite
2633,717650.445706,7.772346e+06,53.0,W,8.0,24.068096,44.197511,24.271859,monzo granite
3131,706073.428899,7.765667e+06,53.0,W,8.0,24.860893,42.062121,22.899788,monzo granite
3277,703216.141426,7.761694e+06,53.0,W,8.0,22.238640,41.690655,25.576752,monzo granite
3321,740293.094230,7.725961e+06,53.0,W,8.0,23.256102,36.570601,24.579421,monzo granite


In [22]:
QAPF_control.to_excel("../_CIPW/CIPW/AREA3c/QAPF_control_points.xlsx", index=True)